In [6]:
import torch
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# variables

EPOCHS = 100
LR = 0.0001
BATCH_SIZE = 1024  # X size (8705536 descriptors, 448 features (from vgg11))
GRADIENT_CLIPPING_VALUE = 0.5
#MODEL_VERSION = 'model_0' if len(os.listdir("./models")) == 0 else f'model_{len(os.listdir("./models"))}'
CHECKPOINT_PATH = "./checkpoints/model_1/model_1_epoch_2000"#f"./checkpoint/{MODEL_VERSION}/{MODEL_VERSION}_epoch_2000"


# loads data and splits into training and testing
X = torch.cat([torch.load(f"../descriptors/sample_{i}") for i in range(int(len(os.listdir("../descriptors")) / 2))], dim=0)
y = torch.cat([torch.load(f"../descriptors/sample_{i}_anotation") for i in range(int(len(os.listdir("../descriptors")) / 2))], dim=0)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y_train = X, y

print(f"X size: {X.size()}")
print(f"y size: {y.size()}")

# makes batchers for training
train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=BATCH_SIZE)

#model definition
model = torch.nn.Sequential(
    torch.nn.Linear(in_features=448, out_features=256),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=256, out_features=128),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=128, out_features=64),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=64, out_features=32),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=32, out_features=1)
)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=LR)

checkpoint = torch.load(CHECKPOINT_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def train_epoch(model, train_loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze(1) 
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIPPING_VALUE)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def evaluate(model, data_loader, loss_fn):
    model.eval()  # change model to evaluation mode
    partial_loss = []
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad(): 
        for X_batch, y_batch in data_loader:
  
            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch)
            partial_loss.append(loss.item())
            total_loss += loss.item() * X_batch.size(0)  

            _, predicted = torch.max(y_pred, 1)
            correct_predictions += (predicted == y_batch).sum().item()
            total_samples += X_batch.size(0)
    
    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    
    return average_loss, accuracy


X size: torch.Size([8718080, 448])
y size: torch.Size([8718080])


In [14]:
#training          
for actual_epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn)
    
    print(f"Epoch {actual_epoch + 1}, Loss: {train_loss}")

    
        

Epoch 1, Loss: 176470.17323072028
Epoch 2, Loss: 176066.82388521446
Epoch 3, Loss: 175636.83740412162
Epoch 4, Loss: 175193.39466863286
Epoch 5, Loss: 174764.5318259198
Epoch 6, Loss: 174374.69021450882
Epoch 7, Loss: 174000.5313901285
Epoch 8, Loss: 173645.1139646337
Epoch 9, Loss: 173278.1671161002
Epoch 10, Loss: 172924.88746365276
Epoch 11, Loss: 172557.23163164392
Epoch 12, Loss: 172175.51114991654
Epoch 13, Loss: 171856.76444436688
Epoch 14, Loss: 171541.23368643684
Epoch 15, Loss: 171220.69418850756
Epoch 16, Loss: 170931.2090096235
Epoch 17, Loss: 170675.60344399078
Epoch 18, Loss: 170440.58496342506
Epoch 19, Loss: 170190.59221080504
Epoch 20, Loss: 169959.12842051726
Epoch 21, Loss: 169744.36028310764
Epoch 22, Loss: 169511.25905780646
Epoch 23, Loss: 169264.60337249635
Epoch 24, Loss: 169026.3517999091
Epoch 25, Loss: 168821.40524118865
Epoch 26, Loss: 168642.43254961798
Epoch 27, Loss: 168482.94100462162
Epoch 28, Loss: 168332.76279160738
Epoch 29, Loss: 168179.03857935878


KeyboardInterrupt: 

In [ ]:
def evaluate(model, data_loader, loss_fn):
    model.eval()  # change model to evaluation mode
    partial_loss = []
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad(): 
        for X_batch, y_batch in data_loader:
  
            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch)
            partial_loss.append(loss.item())
            total_loss += loss.item() * X_batch.size(0)  

            _, predicted = torch.max(y_pred, 1)
            correct_predictions += (predicted == y_batch).sum().item()
            total_samples += X_batch.size(0)
    
    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    
    return partial_loss, average_loss, accuracy